# imports

In [1]:

import pyreadr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
comp_light = (
    pyreadr.read_r("wrds_data/compustat_all_light.rds")[None]
    .assign(
        datadate=lambda x: pd.to_datetime(x.datadate, format="%Y-%m-%d"),
        WCTA = lambda df: df["wcap"] / df["at"],
        RETA = lambda df: df["re"] / df["at"],
        EBTA = lambda df: df["ebit"] / df["at"],
        TLTA = lambda df: df["lt"] / df["at"],
        SLTA = lambda df: df["sale"] / df["at"],
    )
)
crsp_daily = (
    pyreadr.read_r("wrds_data/crsp_daily_light.rds")[None]
    .assign(
        date=lambda x: pd.to_datetime(x.date, format="%Y-%m-%d"),
        linkdt=lambda x: pd.to_datetime(x.linkdt, format="%Y-%m-%d"),
        linkenddt=lambda x: pd.to_datetime(x.linkenddt, format="%Y-%m-%d")
    )
)

company_all = (
    pyreadr.read_r("wrds_data/company_all.rds")[None]
    .assign(
        dldte = lambda x: pd.to_datetime(x.dldte, format="%Y-%m-%d"),
    )
    .query('conm in ("APPLE INC","ENRON CORP","EASTMAN KODAK CO")')
    .filter(["conm","gvkey","dlrsn","dldte","fyrc"])
)

company_variable = (
    pyreadr.read_r("wrds_data/compustat_company_variables.rds")[None]
    .query('variable_postgres in ("dldte","dlrsn")')
)

inact_df = (
    pyreadr.read_r("wrds_data/inact_all.rds")[None]
)

ccmxpf_linktable = (
    pyreadr.read_r("wrds_data/ccmxpf_linktable.rds")[None]
)

In [3]:
print(crsp_daily)

        permno     cusip       date        vol  shrout     prc         cap  \
0      11260.0  17119610 1992-12-28   611700.0  292415  32.750  9576591.25   
1      11260.0  17119610 1992-12-29  1343500.0  292415  32.750  9576591.25   
2      11260.0  17119610 1992-12-30   616500.0  292415  32.750  9576591.25   
3      11260.0  17119610 1992-12-31   952800.0  292840  32.000  9370880.00   
4      11260.0  17119610 1993-01-04  1315000.0  292840  32.625  9553905.00   
...        ...       ...        ...        ...     ...     ...         ...   
37350  92530.0  13134730 2018-03-02  6587358.0  360543  15.220  5487464.46   
37351  92530.0  13134730 2018-03-05  4862799.0  360543  15.220  5487464.46   
37352  92530.0  13134730 2018-03-06  3297139.0  360543  15.240  5494675.32   
37353  92530.0  13134730 2018-03-07  5738378.0  360543  15.240  5494675.32   
37354  92530.0  13134730 2018-03-08  4962299.0  360543  15.250  5498280.75   

        close     low    high     bid     ask    open       ret

# mergeing crsp and compustat data

In [4]:
acc = (
    comp_light
    .query('conm in ("APPLE INC","ENRON CORP","EASTMAN KODAK CO")')
    .filter(['gvkey','fyear','conm','WCTA','RETA','EBTA','TLTA','SLTA'])
)
acc.head()

,gvkey,fyear,conm,WCTA,RETA,EBTA,TLTA,SLTA
0,001690,1980,APPLE INC,0.249824,0.222188,0.360918,0.602923,1.792288
1,001690,1981,APPLE INC,0.615387,0.212174,0.259549,0.303922,1.313709
2,001690,1982,APPLE INC,0.646527,0.324277,0.285670,0.281438,1.629632
3,001690,1983,APPLE INC,0.611247,0.348892,0.232921,0.321029,1.765731
4,001690,1984,APPLE INC,0.548142,0.324064,0.115935,0.411038,1.921784


In [5]:
subset_mapping = (ccmxpf_linktable
 .query(f'gvkey in {acc.gvkey.unique().tolist()}')
 .merge(
    company_all.filter(['gvkey','conm']),
    how='left',
    on='gvkey'
 )
 )
subset_mapping

,permno,gvkey,linkdt,linkenddt,conm
0,14593.0,001690,1980-12-12,2024-10-12,APPLE INC
1,11754.0,004194,1950-01-01,1962-01-30,EASTMAN KODAK CO
2,11754.0,004194,1962-01-31,2012-01-18,EASTMAN KODAK CO
3,14276.0,004194,2013-11-01,2024-10-12,EASTMAN KODAK CO
4,23317.0,006127,1962-01-01,1962-01-30,ENRON CORP
5,23317.0,006127,1962-01-31,2002-01-11,ENRON CORP


In [6]:
# Selecting specific columns from crsp_daily
crsp_daily_selected = crsp_daily[['cusip', 'permno', 'date', 'prc', 'vol', 'shrout', 'bid', 'ask']]

# Merging with subset_mapping using a left join on 'permno'
merged_df = crsp_daily_selected.merge(subset_mapping, on='permno', how='left', indicator=True)

# Filtering rows where 'gvkey' is not null and 'date' is between 'linkdt' and 'linkenddt'
merged_crsp_compustat_sub = merged_df[
    (merged_df['gvkey'].notna()) & (merged_df['date'] >= merged_df['linkdt']) & (merged_df['date'] <= merged_df['linkenddt'])
]

In [7]:
merged_crsp_compustat_sub.head()

,cusip,permno,date,prc,vol,shrout,bid,ask,gvkey,linkdt,linkenddt,conm,_merge
1484,27746110,11754.0,1992-12-28,40.375,415500.0,325036,40.250,40.375,004194,1962-01-31,2012-01-18,EASTMAN KODAK CO,both
1486,27746110,11754.0,1992-12-29,40.375,668600.0,325036,40.250,40.500,004194,1962-01-31,2012-01-18,EASTMAN KODAK CO,both
1488,27746110,11754.0,1992-12-30,40.375,703000.0,325036,40.250,40.500,004194,1962-01-31,2012-01-18,EASTMAN KODAK CO,both
1490,27746110,11754.0,1992-12-31,40.500,529700.0,325215,40.625,40.750,004194,1962-01-31,2012-01-18,EASTMAN KODAK CO,both
1492,27746110,11754.0,1993-01-04,40.875,1238100.0,325215,40.875,41.000,004194,1962-01-31,2012-01-18,EASTMAN KODAK CO,both


# plot

In [9]:
# Calculate market capitalization
import myplotlib.pyplot as plt
merged_crsp_compustat_sub['market_cap'] = merged_crsp_compustat_sub['prc'] * merged_crsp_compustat_sub['shrout'] * 1000

# Plot
plt.figure(figsize=(10, 6))
sns.lineplot(data=merged_crsp_compustat_sub, x='date', y='market_cap', hue='conm')

# Log scale for y-axis
plt.yscale('log')
#plt.gca().yaxis.set_major_formatter(FuncFormatter(short_scale_formatter))
plt.xlabel('Date')
plt.ylabel('Market Capitalization')
plt.title('Market Capitalization Over Time')
plt.legend(title='Company Name')
plt.show()

ModuleNotFoundError: No module named 'myplotlib'

# default data (bankcuptcy)

In [5]:
# Read the Excel file
df = pd.read_excel('/Users/mariegoardet/Desktop/default_data/Florida-UCLA-LoPucki Bankruptcy Research Database 1-12-2023.xlsx')

# Filter columns whose names have more than one character
filtered_columns = [col for col in df.columns if len(col) > 1]
lopucki = df[filtered_columns]

# Display the result
print(lopucki)

                                  NameCorp  \
0                   1st Centennial Bancorp   
1     21st Century Oncology Holdings, Inc.   
2               A. H. Robins Company, Inc.   
3                       A. M. Castle & Co.   
4                       A123 Systems, Inc.   
...                                    ...   
1213      York Research Corporation (2002)   
1214              Young Broadcasting, Inc.   
1215                            Zale Corp.   
1216              Zenith Electronics Corp.   
1217              Ziff Davis Holdings Inc.   

                                          AfterEmerging  Assets1Before  \
0                                                   NaN            NaN   
1                                                   NaN            NaN   
2      Became a unit of American Home Products 12/15/89     648.000000   
3                                                   NaN     329.328000   
4     assets acquired by Wanxiang America Corporatio...     625.902000   
...

In [6]:
lopucki

,NameCorp,AfterEmerging,Assets1Before,Assets2Before,Assets3Before,AssetsBefore,AssetsCurrDollar,AssetsEmerging,AssetsPetCurrDollar,AssetsPetition,...,TurnPerson,TurnRepRange,TurnType,UsPopFiling,Voluntary,YearConfirmed,YearDisposed,YearEmerged,YearFiled,YearsEmergeToRefile
0,1st Centennial Bancorp,NaN,NaN,690.000,NaN,690.000000,963,NaN,NaN,NaN,...,NaN,NaN,NaN,306770000,voluntary,NaN,NaN,NaN,2009,NaN
1,"21st Century Oncology Holdings, Inc.",NaN,NaN,1128.244,NaN,1128.244000,1368,NaN,NaN,NaN,...,NaN,NaN,NaN,325719178,voluntary,2018.0,2018.0,2018.0,2017,NaN
2,"A. H. Robins Company, Inc.",Became a unit of American Home Products 12/15/89,648.000000,509.663,439.983,648.000000,1781,NaN,1281.0,466.000000,...,NaN,NaN,NaN,237920000,voluntary,1988.0,1988.0,1988.0,1985,NaN
3,A. M. Castle & Co.,NaN,329.328000,NaN,NaN,329.328000,399,332.285,NaN,NaN,...,NaN,NaN,NaN,325719178,voluntary,2017.0,2017.0,2017.0,2017,NaN
4,"A123 Systems, Inc.",assets acquired by Wanxiang America Corporatio...,625.902000,NaN,NaN,625.902000,803,NaN,590.0,459.795000,...,none,NaN,none,313910000,voluntary,2013.0,2013.0,2012.0,2012,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,York Research Corporation (2002),NaN,263.451157,NaN,NaN,263.451157,435,NaN,198.0,119.900326,...,NaN,NaN,NaN,287630000,involuntary,2002.0,2002.0,2002.0,2001,NaN
1214,"Young Broadcasting, Inc.",reverse merged with Media General 11/2013,348.222526,NaN,NaN,348.222526,487,NaN,805.0,575.600000,...,David Pauker,yes,manage,306770000,voluntary,2010.0,2010.0,2010.0,2009,NaN
1215,Zale Corp.,Prosperous 2001,1789.178000,2013.247,1450.000,1789.178000,3845,1013.523,3820.0,1777.500000,...,NaN,NaN,NaN,256510000,involuntary,1993.0,1993.0,1993.0,1992,NaN
1216,Zenith Electronics Corp.,"Acquired by LGE at confirmation, subsidiary, w...",350.000000,NaN,NaN,350.000000,622,148.300,NaN,NaN,...,NaN,NaN,NaN,279040000,voluntary,1999.0,1999.0,1999.0,1999,NaN


## Market data : BDD ALL (échec)

In [4]:
crsp_daily_all = (
    pyreadr.read_r("/Users/mariegoardet/Desktop/wrds_data/crsp_daily_full.rds")[None]
    .assign(
        date=lambda x: pd.to_datetime(x.date, format="%Y-%m-%d"),
        linkdt=lambda x: pd.to_datetime(x.linkdt, format="%Y-%m-%d"),
        linkenddt=lambda x: pd.to_datetime(x.linkenddt, format="%Y-%m-%d")
    )
)
crsp_daily_all.head()

AttributeError: 'DataFrame' object has no attribute 'linkdt'

In [8]:
# Charger le fichier en entier (peut être lent si très gros)
crsp_daily_all = pyreadr.read_r("/Users/mariegoardet/Desktop/wrds_data/crsp_daily_full.rds")[None]

# Sélectionner seulement le premier tiers
n_rows = len(crsp_daily_all)  # Nombre total de lignes
first_third = crsp_daily_all.iloc[: n_rows // 3]  # Garder le premier tiers

# Convertir les dates
first_third = first_third.assign(
    date=lambda x: pd.to_datetime(x.date, format="%Y-%m-%d"),
    linkdt=lambda x: pd.to_datetime(x.linkdt, format="%Y-%m-%d"),
    linkenddt=lambda x: pd.to_datetime(x.linkenddt, format="%Y-%m-%d")
)

# Afficher un aperçu
first_third.head()

AttributeError: 'DataFrame' object has no attribute 'linkdt'

## Market data : passer de daily à year (light)

On a deja importé crsp_daily plus haut mais je remets le code en commentaire au cas où :

In [10]:
crsp_daily = (
    pyreadr.read_r("wrds_data/crsp_daily_light.rds")[None]
    .assign(
        date=lambda x: pd.to_datetime(x.date, format="%Y-%m-%d"),
       linkdt=lambda x: pd.to_datetime(x.linkdt, format="%Y-%m-%d"),
       linkenddt=lambda x: pd.to_datetime(x.linkenddt, format="%Y-%m-%d")
    )
)

- cusip : Identifiant unique de 9 caractères attribué aux titres financiers aux États-Unis. Il permet d'identifier de manière unique une action ou une obligation.
- date : Date de l'observation du titre.
- vol : Volume des actions échangées pendant la journée.
- shrout : Nombre total d'actions en circulation (Shares Outstanding).
- prc : Prix de clôture ajusté du titre. Peut être négatif si c'est un prix "bid/ask". 
- cap : Capitalisation boursière du titre, calculée comme prc * shrout.
- close : Prix de clôture du titre (non ajusté).
- low : Prix le plus bas atteint pendant la journée.
- high : Prix le plus haut atteint pendant la journée.
- bid : Dernier prix d'achat enregistré (bid price).
- ask : Dernier prix de vente enregistré (ask price).
- open : Prix d'ouverture du titre.
- ret : Rendement journalier du titre (return).
- gvkey : Identifiant unique attribué par Compustat pour suivre une entreprise dans la base de données.
- linkdt : Date de début de la liaison entre CRSP et Compustat.
- linkenddt : Date de fin de la liaison entre CRSP et Compustat (si applicable, sinon c'est une valeur manquante).
- conm : Nom de l'entreprise associée au titre

In [11]:
crsp_daily

,permno,cusip,date,vol,shrout,prc,cap,close,low,high,bid,ask,open,ret,gvkey,linkdt,linkenddt,conm
0,11260.0,17119610,1992-12-28,611700.0,292415,32.750,9576591.25,32.750,31.625,32.750,32.500,32.875,31.625,0.031496,003022,1962-01-31,1998-11-30,CHRYSLER CORP
1,11260.0,17119610,1992-12-29,1343500.0,292415,32.750,9576591.25,32.750,32.750,33.500,32.750,32.875,32.875,0.000000,003022,1962-01-31,1998-11-30,CHRYSLER CORP
2,11260.0,17119610,1992-12-30,616500.0,292415,32.750,9576591.25,32.750,32.500,33.000,32.625,32.750,32.875,0.000000,003022,1962-01-31,1998-11-30,CHRYSLER CORP
3,11260.0,17119610,1992-12-31,952800.0,292840,32.000,9370880.00,32.000,31.500,32.750,31.750,32.125,32.375,-0.022901,003022,1962-01-31,1998-11-30,CHRYSLER CORP
4,11260.0,17119610,1993-01-04,1315000.0,292840,32.625,9553905.00,32.625,31.750,32.625,32.500,32.750,31.875,0.019531,003022,1962-01-31,1998-11-30,CHRYSLER CORP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37350,92530.0,13134730,2018-03-02,6587358.0,360543,15.220,5487464.46,15.220,15.220,15.230,15.220,15.230,15.220,0.000000,063605,2008-02-07,2018-03-29,CALPINE CORP
37351,92530.0,13134730,2018-03-05,4862799.0,360543,15.220,5487464.46,15.220,15.220,15.230,15.220,15.230,15.220,0.000000,063605,2008-02-07,2018-03-29,CALPINE CORP
37352,92530.0,13134730,2018-03-06,3297139.0,360543,15.240,5494675.32,15.240,15.220,15.240,15.230,15.240,15.220,0.001314,063605,2008-02-07,2018-03-29,CALPINE CORP
37353,92530.0,13134730,2018-03-07,5738378.0,360543,15.240,5494675.32,15.240,15.230,15.240,15.230,15.240,15.240,0.000000,063605,2008-02-07,2018-03-29,CALPINE CORP


Comment agréger chaque variable ? 
1) On créer les variables : 

- var_prc :  par gvkey par année en se servant de la premiere date et derniere date de chaque gvkey, on obient une var_prc pour chaque  année de chaque gvkey. 
- taux_evol_prc : en se servant de la premiere date et derniere date (min date et max date PAR ANNEE) par année par gvkey. derniere-premiere/premiere. 

2) On groupe les autres variables selon les conditions suivantes :  

- vol (volume) → Somme (sum), car on veut le total des actions échangées sur l’année.
- shrout (actions en circulation) → Moyenne (mean), car ce nombre évolue dans le temps 
- cap (capitalisation boursière) → Moyenne (mean), car elle fluctue sur l’année.
- close (prix non ajusté) → Dernier prix de l’année (last).
- low (prix le plus bas) → Minimum de l’année (min).
- high (prix le plus haut) → Maximum de l’année (max).
- bid (dernier prix d’achat) → Dernière valeur (last).
- ask (dernier prix de vente) → Dernière valeur (last).
- open (prix d’ouverture) → Premier prix enregistré de l’année (first).
- ret (rendement) → Moyenne des rendements journaliers (mean).


- gvkey → Premier identifiant de l’année (first).
- cusip → Premier identifiant de l’année (first).
- conm → Premier nom enregistré de l’année (first).


- linkdt (début de liaison) → Première date enregistrée (min).
- linkenddt (fin de liaison) → Dernière date enregistrée (max).



Etape 1 : créer les nouvelles variables

1) Variance du prc par année par entreprise

In [12]:
crsp_daily['date'] = pd.to_datetime(crsp_daily['date'])
crsp_daily['year'] = crsp_daily['date'].dt.year
crsp_daily['var_prc'] = crsp_daily.groupby(['gvkey', 'year'])['prc'].transform('var')
crsp_daily.head()

,permno,cusip,date,vol,shrout,prc,cap,close,low,high,bid,ask,open,ret,gvkey,linkdt,linkenddt,conm,year,var_prc
0,11260.0,17119610,1992-12-28,611700.0,292415,32.750,9576591.25,32.750,31.625,32.750,32.500,32.875,31.625,0.031496,003022,1962-01-31,1998-11-30,CHRYSLER CORP,1992,0.140625
1,11260.0,17119610,1992-12-29,1343500.0,292415,32.750,9576591.25,32.750,32.750,33.500,32.750,32.875,32.875,0.000000,003022,1962-01-31,1998-11-30,CHRYSLER CORP,1992,0.140625
2,11260.0,17119610,1992-12-30,616500.0,292415,32.750,9576591.25,32.750,32.500,33.000,32.625,32.750,32.875,0.000000,003022,1962-01-31,1998-11-30,CHRYSLER CORP,1992,0.140625
3,11260.0,17119610,1992-12-31,952800.0,292840,32.000,9370880.00,32.000,31.500,32.750,31.750,32.125,32.375,-0.022901,003022,1962-01-31,1998-11-30,CHRYSLER CORP,1992,0.140625
4,11260.0,17119610,1993-01-04,1315000.0,292840,32.625,9553905.00,32.625,31.750,32.625,32.500,32.750,31.875,0.019531,003022,1962-01-31,1998-11-30,CHRYSLER CORP,1993,34.313600


2) Taux évolution du prc par année par entreprise

In [6]:
crsp_daily['date'] = pd.to_datetime(crsp_daily['date']) 
crsp_daily['year'] = crsp_daily['date'].dt.year  

# Trouver les premières et dernières dates de chaque année pour chaque entreprise
first_last_dates = crsp_daily.groupby(['gvkey', 'year']).agg(
    first_date=('date', 'min'), 
    last_date=('date', 'max')
).reset_index()

print(first_last_dates)




      gvkey  year first_date  last_date
0    001690  1992 1992-06-15 1992-12-31
1    001690  1993 1993-01-04 1993-12-31
2    001690  1994 1994-01-03 1994-12-30
3    001690  1995 1995-01-03 1995-12-29
4    001690  1996 1996-01-02 1996-12-31
..      ...   ...        ...        ...
159  063605  2014 2014-01-02 2014-12-31
160  063605  2015 2015-01-02 2015-12-31
161  063605  2016 2016-01-04 2016-12-30
162  063605  2017 2017-01-03 2017-12-29
163  063605  2018 2018-01-02 2018-03-08

[164 rows x 4 columns]


In [7]:
# Associer les prix `prc` aux dates `first_date` et `last_date`
first_prices = crsp_daily[['gvkey', 'year', 'date', 'prc']].merge(
    first_last_dates[['gvkey', 'year', 'first_date']], 
    left_on=['gvkey', 'year', 'date'], 
    right_on=['gvkey', 'year', 'first_date'], 
    how='inner'
)[['gvkey', 'year', 'prc']].rename(columns={'prc': 'first_prc'})

last_prices = crsp_daily[['gvkey', 'year', 'date', 'prc']].merge(
    first_last_dates[['gvkey', 'year', 'last_date']], 
    left_on=['gvkey', 'year', 'date'], 
    right_on=['gvkey', 'year', 'last_date'], 
    how='inner'
)[['gvkey', 'year', 'prc']].rename(columns={'prc': 'last_prc'})

# Fusionner `first_prc` et `last_prc` avec `first_last_dates`
first_last_dates = first_last_dates.merge(first_prices, on=['gvkey', 'year'], how='left')
first_last_dates = first_last_dates.merge(last_prices, on=['gvkey', 'year'], how='left')

first_last_dates.head()


,gvkey,year,first_date,last_date,first_prc,last_prc
0,001690,1992,1992-06-15,1992-12-31,52.625,59.750
1,001690,1993,1993-01-04,1993-12-31,58.250,29.250
2,001690,1994,1994-01-03,1994-12-30,29.875,39.000
3,001690,1995,1995-01-03,1995-12-29,38.375,31.875
4,001690,1996,1996-01-02,1996-12-31,32.125,20.875


In [10]:
#Ajouter le taux evol prc 
first_last_dates['tx_evol_prc'] = (first_last_dates['last_prc'] - first_last_dates['first_prc']) / first_last_dates['first_prc']

first_last_dates.head()

,gvkey,year,first_date,last_date,first_prc,last_prc,tx_evol_prc
0,001690,1992,1992-06-15,1992-12-31,52.625,59.750,0.135392
1,001690,1993,1993-01-04,1993-12-31,58.250,29.250,-0.497854
2,001690,1994,1994-01-03,1994-12-30,29.875,39.000,0.305439
3,001690,1995,1995-01-03,1995-12-29,38.375,31.875,-0.169381
4,001690,1996,1996-01-02,1996-12-31,32.125,20.875,-0.350195


Etape 2 : aggrégation par année et ajout des variables 

In [13]:

aggregations = {
    'vol': 'sum',         # Volume total échangé sur l’année
    'shrout': 'mean',     # Moyenne des actions en circulation
    'cap': 'mean',        # Capitalisation moyenne sur l’année
    'close': 'last',      # Dernier prix non ajusté
    'low': 'min',         # Prix le plus bas atteint sur l’année
    'high': 'max',        # Prix le plus haut atteint sur l’année
    'bid': 'last',        # Dernière valeur de bid
    'ask': 'last',        # Dernière valeur de ask
    'open': 'first',      # Premier prix d’ouverture de l’année
    'ret': 'mean',        # Rendement moyen
    'cusip': 'first',     # Premier cusip enregistré
    'conm': 'first',      # Premier nom d’entreprise enregistré
    'linkdt': 'min',      # Première date de liaison
    'linkenddt': 'max',    # Dernière date de liaison
    'var_prc':'mean',
    'prc':'std'
}

# 🔹 Grouper par entreprise (`gvkey`) et année (`year`)
crsp_yearly = crsp_daily.groupby(['gvkey', 'year'], as_index=False).agg(aggregations)


crsp_yearly.head()

,gvkey,year,vol,shrout,cap,close,low,high,bid,ask,open,ret,cusip,conm,linkdt,linkenddt,var_prc,prc
0,001690,1992,205326069.0,118660.835714,5.882697e+06,59.750,41.500,61.250,59.50,59.750,54.000,0.000931,03783310,APPLE INC,1980-12-12,2024-10-10,29.031716,5.388109
1,001690,1993,507669464.0,116515.557312,4.789324e+06,29.250,22.000,65.250,29.25,29.500,59.500,-0.002260,03783310,APPLE INC,1980-12-12,2024-10-10,175.499851,13.247636
2,001690,1994,508924449.0,118130.008000,4.022365e+06,39.000,24.625,43.750,38.75,39.000,29.500,0.001377,03783310,APPLE INC,1980-12-12,2024-10-10,19.179991,4.379497
3,001690,1995,660486325.0,121476.633466,4.920488e+06,31.875,31.625,50.125,31.75,31.875,38.875,-0.000684,03783310,APPLE INC,1980-12-12,2024-10-10,16.576081,4.071373
4,001690,1996,477850278.0,123786.063241,3.082474e+06,20.875,16.000,35.500,20.75,20.875,32.250,-0.001120,03783310,APPLE INC,1980-12-12,2024-10-10,10.704418,3.271761


In [14]:

crsp_merged = crsp_yearly.merge(first_last_dates, on=['gvkey', 'year'], how='left')
crsp_merged.head()


NameError: name 'first_last_dates' is not defined

## Fonction pour filtrer le data set crsp_daily entre deux dates

On reprend les données de marchés initiales qu'on va grouper par gvkey (et pas par year)

In [28]:
crsp_daily.head()
    

,permno,cusip,date,vol,shrout,prc,cap,close,low,high,bid,ask,open,ret,gvkey,linkdt,linkenddt,conm,year,var_prc
0,11260.0,17119610,1992-12-28,611700.0,292415,32.750,9576591.25,32.750,31.625,32.750,32.500,32.875,31.625,0.031496,003022,1962-01-31,1998-11-30,CHRYSLER CORP,1992,0.140625
1,11260.0,17119610,1992-12-29,1343500.0,292415,32.750,9576591.25,32.750,32.750,33.500,32.750,32.875,32.875,0.000000,003022,1962-01-31,1998-11-30,CHRYSLER CORP,1992,0.140625
2,11260.0,17119610,1992-12-30,616500.0,292415,32.750,9576591.25,32.750,32.500,33.000,32.625,32.750,32.875,0.000000,003022,1962-01-31,1998-11-30,CHRYSLER CORP,1992,0.140625
3,11260.0,17119610,1992-12-31,952800.0,292840,32.000,9370880.00,32.000,31.500,32.750,31.750,32.125,32.375,-0.022901,003022,1962-01-31,1998-11-30,CHRYSLER CORP,1992,0.140625
4,11260.0,17119610,1993-01-04,1315000.0,292840,32.625,9553905.00,32.625,31.750,32.625,32.500,32.750,31.875,0.019531,003022,1962-01-31,1998-11-30,CHRYSLER CORP,1993,34.313600


In [15]:
def filtrer_crsp(start_date, end_date):
    crsp_daily['date'] = pd.to_datetime(crsp_daily['date'])
    crsp_daily_filtered=crsp_daily[(crsp_daily['date'] >= start_date) & (crsp_daily['date'] < end_date)]

    aggregations = {
    'vol': 'sum',         # Volume total échangé par gvkey 
    'shrout': 'mean',     # Moyenne des actions en circulation par gvkey 
    'cap': 'mean',        # Capitalisation moyenne par gvkey
    'close': 'last',      # Dernier prix non ajusté par gvkey
    'low': 'min',         # Prix le plus bas atteint par gvkey
    'high': 'max',        # Prix le plus haut atteint par gvkey
    'bid': 'last',        # Dernière valeur de bid par gvkey
    'ask': 'last',        # Dernière valeur de ask par gvkey
    'open': 'first',      # Premier prix d’ouverture de l’année
    'ret': 'mean',        # Rendement moyen par gvkey
    'cusip': 'first',     # Premier cusip enregistré par gvkey
    'conm': 'first',      # Premier nom d’entreprise enregistré 
    'linkdt': 'min',      # Première date de liaison par gvkey
    'linkenddt': 'max',    # Dernière date de liaison par gvkey 
    'prc':'std'             #std du prc par gvkey 
    }   
   
    # 🔹 Grouper par entreprise (`gvkey`) et année (`year`)
    crsp_per_gvkey = crsp_daily_filtered.groupby(['gvkey'], as_index=False).agg(aggregations)

    #calcul du taux d'évolution du prc entre la strat

    # Trouver les premières et dernières dates pour chaque entreprise
    first_last_dates = crsp_daily_filtered.groupby(['gvkey']).agg(
        first_date=('date', 'min'), 
        last_date=('date', 'max')   
    ).reset_index()
    
   # Associer les prix `prc` aux dates `first_date` et `last_date`
    first_prices = crsp_daily_filtered[['gvkey', 'date', 'prc']].merge(
        first_last_dates[['gvkey', 'first_date']], 
        left_on=['gvkey', 'date'], 
        right_on=['gvkey', 'first_date'], 
        how='inner'
    )[['gvkey', 'prc']].rename(columns={'prc': 'first_prc'})
    last_prices = crsp_daily_filtered[['gvkey', 'date', 'prc']].merge(
        first_last_dates[['gvkey', 'last_date']], 
        left_on=['gvkey', 'date'], 
        right_on=['gvkey', 'last_date'], 
        how='inner'
    )[['gvkey', 'prc']].rename(columns={'prc': 'last_prc'})

    # Fusionner `first_prc` et `last_prc` avec `first_last_dates`
    first_last_dates = first_last_dates.merge(first_prices, on=['gvkey'], how='left')
    first_last_dates = first_last_dates.merge(last_prices, on=['gvkey'], how='left')    
    #Ajouter le taux evol prc 
    first_last_dates['tx_evol_prc'] = (first_last_dates['last_prc'] - first_last_dates['first_prc']) / first_last_dates['first_prc']
    
    crsp_per_gvkey= crsp_per_gvkey.merge(first_last_dates, on=['gvkey'], how='left')

    return crsp_per_gvkey


Attention : penser à mettre dates entre guillemets lol

In [16]:
start_date = '1992-12-28'
end_date = '1993-01-04'
crsp_result = filtrer_crsp(start_date, end_date)

In [17]:
crsp_result.head()

,gvkey,vol,shrout,cap,close,low,high,bid,ask,open,...,cusip,conm,linkdt,linkenddt,prc,first_date,last_date,first_prc,last_prc,tx_evol_prc
0,001690,3398634.0,117841.00,7.000492e+06,59.750,58.750,60.750,59.500,59.750,59.250,...,03783310,APPLE INC,1980-12-12,2024-10-10,0.449247,1992-12-28,1992-12-31,59.500,59.750,0.004202
1,003022,3524500.0,292521.25,9.525163e+06,32.000,31.500,33.500,31.750,32.125,31.625,...,17119610,CHRYSLER CORP,1962-01-31,1998-11-30,0.375000,1992-12-28,1992-12-31,32.750,32.000,-0.022901
2,004194,2316800.0,325080.75,1.313530e+07,40.500,40.125,40.875,40.625,40.750,40.375,...,27746110,EASTMAN KODAK CO,1962-01-31,2012-01-18,0.062500,1992-12-28,1992-12-31,40.375,40.500,0.003096
3,005073,6295700.0,693439.00,2.260043e+07,32.250,32.125,33.375,32.250,32.375,33.000,...,37044210,GENERAL MOTORS CO,1962-01-31,2009-06-01,0.373260,1992-12-28,1992-12-31,33.125,32.250,-0.026415
4,006127,685300.0,111551.00,5.272984e+06,46.375,46.375,48.000,46.125,46.625,47.875,...,29356110,ENRON CORP,1962-01-31,2002-01-11,0.672178,1992-12-28,1992-12-31,48.000,46.375,-0.033854


In [18]:
crsp_for_model =crsp_result.drop(columns=['linkdt', 'linkenddt','first_prc','last_prc','first_date','last_date','cusip'])
crsp_for_model.head()

,gvkey,vol,shrout,cap,close,low,high,bid,ask,open,ret,conm,prc,tx_evol_prc
0,001690,3398634.0,117841.00,7.000492e+06,59.750,58.750,60.750,59.500,59.750,59.250,0.003231,APPLE INC,0.449247,0.004202
1,003022,3524500.0,292521.25,9.525163e+06,32.000,31.500,33.500,31.750,32.125,31.625,0.002149,CHRYSLER CORP,0.375000,-0.022901
2,004194,2316800.0,325080.75,1.313530e+07,40.500,40.125,40.875,40.625,40.750,40.375,-0.000764,EASTMAN KODAK CO,0.062500,0.003096
3,005073,6295700.0,693439.00,2.260043e+07,32.250,32.125,33.375,32.250,32.375,33.000,-0.005693,GENERAL MOTORS CO,0.373260,-0.026415
4,006127,685300.0,111551.00,5.272984e+06,46.375,46.375,48.000,46.125,46.625,47.875,-0.007879,ENRON CORP,0.672178,-0.033854
